# Iterating On Stop Assignment

## Loading data

In [1]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import pandas as pd
from operator import itemgetter
from math import sqrt, cos
from time import time, mktime
import folium
from folium.plugins import TimestampedGeoJson
import datetime
import numpy as np

In [3]:
# Muting chained assignment warning; needs refactoring
# but doesn't affect performance

pd.set_option('mode.chained_assignment', None)

### A single day's (05/24/2020) full data pulled from api

In [4]:
url = 'http://sfmta-ds.eba-hqpuyrup.us-east-1.elasticbeanstalk.com/daily-general-json'

In [5]:
json_data = requests.get(url, params={'day': '2020-05-24'}).json()

In [6]:
# making df

full_data = pd.DataFrame(data=json_data).sort_values('timestamp')

In [7]:
# sub dfs for testing
# single route, all vehicles
# single route, single vehicle

nbus = full_data[full_data['rid']=='NBUS']
nbus_highest = nbus[nbus['vid']==(nbus['vid'].value_counts().index[0])].sort_values('timestamp')

In [8]:
# using stops gathered by Labs 22 for expediency
# all stops
# all stops in one route

stops = pd.read_csv('https://raw.githubusercontent.com/Lambda-School-Labs/sfmta-data-analysis-ds/master/deprecated_assets/datasets/route_info.csv')
nbus_stops = stops[stops['route_id']=='NBUS']

## Engineering Probable Nearest Stop and Distance (For Confidence)

### Original helper function for wrangling

In [9]:
def wrangle_bus(df):
    """
    preps dataframe for a single bus
    gives accurate timestamps and naively calculates 
    dwell time as 1min per checkin with motion (kph <= 0)
    
    Largest bottleneck for time-cost in df prep
    currently not implemented until refactor
    """
    
    times = df['timestamp'].values
    ages = df['age'].values
    
    df['adjusted_timestamp'] = [pd.Timestamp(times[x]) - pd.Timedelta(seconds=ages[x]) for 
                                x in range(len(df['timestamp']))]
    
    df['timestamp'] = [pd.Timestamp(times[x]) for x in range(len(df['timestamp']))]

    dwell_count = 0
    dwell_totals = []

    for x in df['kph']:
        if x <= 0:
          dwell_count += 1
          dwell_totals.append(dwell_count)
        elif x > 0:
          dwell_totals.append(0)
          dwell_count = 0
            
    df['dwell'] = [dwell_totals[x] for x in range(len(df))]

    return df

### Function to calculate nearest stop within $X$ km by projected euclidean distance

In [10]:
def fcc_projection(loc1, loc2):
    """
    function to apply FCC recommended formulae
    for calculating distances on earth projected to a plane
    
    significantly faster computationally, negligible loss in accuracy
    
    Args: 
    loc1 - a tuple of lat/lon
    loc2 - a tuple of lat/lon
    """
    lat1, lat2 = loc1[0], loc2[0]
    lon1, lon2 = loc1[1], loc2[1]
    
    mean_lat = (lat1+lat2)/2
    delta_lat = lat2 - lat1
    delta_lon = lon2 - lon1
    
    k1 = 111.13209 - 0.56605*cos(2*mean_lat) + .0012*cos(4*mean_lat)
    k2 = 111.41513*cos(mean_lat) - 0.09455*cos(3*mean_lat) + 0.00012*cos(5*mean_lat)
    
    distance = sqrt((k1*delta_lat)**2 + (k2*delta_lon)**2)
    
    return distance

In [11]:
def assign_stop(df, stops):
    """
    applies basic wrangling function
    calculates nearest stop from reported location in km
    returns dataframe with reported location, 
    nearest stop (coords and name), and distance between

    tested with single buses on single routes on a single day;
    technically route/vehicle/time agnostic
    don't foresee any issues generalizing
    
    implements FCC projection formulae for calculating distance
    
    Args:
    df - dataframe of transit data, requires 'latitude', 'longitude' columns
    stops - datafram of stops data, requires 'lat', 'lon', 'title' columns
    """
    
    # TO-DO: error handling for missing routes from either df or stops
    # Currently handling by intersecting sets during function call
    
    start = time()
    
    # wrangle_bus function is now largest time bottleneck - may be removed
    wrangle_bus(df)

    # creating list of lat/lon dictionaries for stops and reported bus locations
    stop_lats = stops['lat'].values
    stop_lons = stops['lon'].values

    reported_lats = df['latitude'].values
    reported_lons = df['longitude'].values

    stop_points = [{'latitude': stop_lats[x], 'longitude': stop_lons[x]} 
                 for x in range(len(stops))]

    reported_points = [{'latitude': reported_lats[x], 
                      'longitude': reported_lons[x]} 
                     for x in range(len(df))]

    # to minimize possible overlap between probable stops
    # 500 ft as km
    # upper end of previous range for minimum distance between stops according to sfmta
    # this value seems good but could use more testing
    radius = .1524

    # dict to tuples to play nice with geopy
    stop_point_tuples = [tuple(stop_points[x].values()) 
                       for x in range(len(stop_points))]

    reported_point_tuples = [tuple(reported_points[x].values()) 
                           for x in range(len(reported_points))]

    df['reported_location'] = reported_point_tuples
    
    print('Prep Complete')
    
    # generating ((lat/lon), distance) tuples for nearest stop within range
    # using FCC ellipsoidal earth projection
    distances = [{x: fcc_projection(location, x) 
                 for x in stop_point_tuples} 
                 for location in reported_point_tuples]
    
    print(f'Distances Generated => {len(distances)}')
    
    # sorting for nearest stop
    distances_sorted = [{k: v for k, v in sorted(distances[x].items(), 
                                                 key=itemgetter(1))}
                       for x in range(len(distances))]
    
    print(f'Distances Sorted => {len(distances_sorted)}')
    
    # creating list of nearest stops
    # nearest stop if nearest stop within radius, else None
    point_stops = [next(iter(distances_sorted[x].items())) 
                   if next(iter(distances_sorted[x].items()))[1] <= radius 
                   else None
                   for x in range(len(distances_sorted))]
    
    print(f'Stops Created => {len(point_stops)}')
    
    # assigning stop name from stops table based on lat/lon from previous step
    stop_tuples = list(zip(stops['lat'], stops['lon']))
    stop_titles = [stops['title'].iloc[stop_tuples.index(stop[0])] 
                   if stop != None
                   else None 
                   for stop in point_stops]
    
    print(f'Titles Created => {len(stop_titles)}')
    
    # pulling lat/lon and distance from tuples for df
    df['nearest_stop'] = [x[0] if x != None else None for x in point_stops]
    df['distance_in_km'] = [x[1] if x != None else None for x in point_stops]

    # pulling stop names from list for df
    df['stops'] = stop_titles

    # dropping columns of redundant information
    df = df.drop(columns=['age', 'rid', 'vid', 'latitude', 'longitude'])
    end = time()
    
    print(f'DF Complete\nTime Elapsed: {end-start} seconds\n')
    
    return df

### Full Day - Single Route, Single Bus

In [12]:
nbus_highest_wrangled = assign_stop(nbus_highest, nbus_stops)

Prep Complete
Distances Generated => 979
Distances Sorted => 979
Stops Created => 979
Titles Created => 979
DF Complete
Time Elapsed: 0.356719970703125 seconds



## Binning Avg. Bus Locations by Time (5 min)

Below functions are predicated on data wrangled by above functions;\
specifically, they depend on the adjusted_timestamp and reported_location.

BIG SPAGHETTTT in here

In [13]:
def time_wrangling(df, delta=5):
    
    delta = datetime.timedelta(minutes=delta)
    
    df['unix_timestamp'] = df['adjusted_timestamp'].apply(lambda d: 
                                                          mktime(d.timetuple()))
    
    df = df.set_index('adjusted_timestamp')
    
    bins = np.arange(min(df['unix_timestamp']),
                     max(df['unix_timestamp'])
                     + delta.seconds, delta.seconds)
    
    binned_df = df.groupby(df['unix_timestamp']
                           .map(lambda t: datetime.datetime.fromtimestamp(bin_from_timestamp(t, bins))))
    
    binned_locs = {str(x): binned_df.get_group(str(x))['reported_location'] for x, y in binned_df}
    
    avg_locs = {x: avg_coords(binned_locs[x][:])
                for x in binned_locs}
    
    return avg_locs

In [14]:
def bin_from_timestamp(timestamp, bins):

    diffs = [abs(timestamp - bin) for bin in bins]
    return bins[diffs.index(min(diffs))]

In [15]:
def avg_coords(coords):
    avg = tuple(map(lambda y: sum(y) / float(len(y)), zip(*coords)))
    return avg

## Mapping!

### Creating GeoJson for playback of timestamped locations

In [16]:
def create_geojson_features(time_locs):
    print('> Creating GeoJSON features...')
    features = []
    for entry in time_locs:
        feature = {
            'type': 'Feature',
            'geometry': {
                'type':'Point', 
                'coordinates':[time_locs[entry][1],time_locs[entry][0]]
            },
            'properties': {
                'time': entry.__str__(),
                'style': {'color' : 'blue'},
                'icon': 'circle',
                'iconstyle':{
                    'fillColor': 'blue',
                    'fillOpacity': 0.8,
                    'stroke': 'true',
                    'radius': 7
                }
            }
        }
        features.append(feature)
    return features

### Defining map and geojson params for playback

In [17]:
def make_map(features, delta, duration):
    print('> Making map...')
    bus_map = folium.Map(location=[37.77293534218353, -122.44596170151804], 
                         control_scale=True, 
                         zoom_start=14)

    TimestampedGeoJson(
        {'type': 'FeatureCollection',
        'features': features},
        period=f'PT{delta}M',
        add_last_point=True,
        auto_play=False,
        loop=False,
        max_speed=2,
        loop_button=True,
        date_options='HH:mm:ss',
        time_slider_drag_update=True,
        duration=f'PT{duration}M'
    ).add_to(bus_map)
    print('> Done.')
    return bus_map

def plot_map(time_locs, delta=5, duration=5):
    features = create_geojson_features(time_locs)
    return make_map(features, delta, duration)

### It's a map!

In [18]:
plot_map(time_wrangling(nbus_highest_wrangled, delta=5), delta=5, duration=5)

> Creating GeoJSON features...
> Making map...
> Done.


In [19]:
# saving map

nbus_single_map = plot_map(time_wrangling(nbus_highest_wrangled, delta=5), delta=5, duration=5)
nbus_single_map.save('nbus_single.html')

> Creating GeoJSON features...
> Making map...
> Done.


## Moving Forward

The above is a very rough-and-ready approach to mapping, but Folium is a fairly powerful and flexible package\
for mapping, and offers plug-and-play for timestamped heatmapping as well.

The immediate issue with above is that currently my code only handles single vehicles. It needs refactoring\
to be able to handle multiple vehicles on the same route. I imagine that process would also make it viable for\
mapping multiple routes (maybe useful), all routes for a transit type (maybe not useful), and all transit (not useful?) ¯\\_(ツ)_/¯

However I think the general direction is worth pursuing; we can generate these maps as a part of our daily report,\
with flexibilty of possibly allowing user input as far as timedeltas, visibility durations, etc. on reports generated\
at will, and FE can serve them as ebmedded html. (Something something iframes?)

If it ends up being better for Web to own map generation, this at least gives a foundation for the data\
we actually have to serve for them to be able to do that, and a quick platform for testing.

While rendering all vehicles on a route (or even generating the right data for such) is currently broken,\
I did do a small amount of testing as far as execution time goes on full-route data;\
assuming I don't do a completely horrific job during refactor, generating these maps or the data for them\
should be a negligble part of daily report generation.